## Mount Drive

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Get required packages

In [0]:
!pip install setuptools --upgrade
!pip install tb-nightly
!git clone https://github.com/ViiSkor/keras-buoy

## Imports and helper functions

In [0]:
import sys
sys.path.extend(['/content/drive/My Drive/VolumMedSeg', '/content/drive/My Drive/VolumMedSeg/src', '/content/keras-buoy/src'])

import os
import random
import pickle
import numpy as np
import nibabel as nib
from keras_buoy.models import ResumableModel
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from matplotlib import animation, rc
from matplotlib import pyplot as plt
from keras_buoy.models import ResumableModel

from src import utils
from src.metrics import dice_coefficient, mean_iou
from src.losses import soft_dice_loss
from src.unet import Unet

rc('animation', html='jshtml')

## Loading Data


### Load 3D data paths


In [0]:
data3D_dir = '/content/drive/My Drive/BTATS2019_preprocessed/3D'
data3D_paths = utils.get_fpaths(data3D_dir)
train_data3D_paths, test_data3D_paths, val_data3D_paths = utils.get_dataset_split(data3D_paths, seed=42)

In [0]:
print(f"Train set has {len(train_data3D_paths)} samples")
print(f"Test set has {len(test_data3D_paths)} samples")
print(f"Validation set has {len(val_data3D_paths)} samples")

Train set has 234 samples
Test set has 50 samples
Validation set has 51 samples


### Load 2D data paths

In [0]:
data2D_dir = '/content/drive/My Drive/BTATS2019_preprocessed/2D/'

In [0]:
train_data2D_paths = utils.get_data_paths4existing_slit(data2D_dir, train_data3D_paths, mode="2D")
test_data2D_paths = utils.get_data_paths4existing_slit(data2D_dir, test_data3D_paths, mode="2D")
val_data2D_paths = utils.get_data_paths4existing_slit(data2D_dir, val_data3D_paths, mode="2D")

In [0]:
train_data2D_paths_unnpacked = utils.unpack_2D_fpaths(train_data2D_paths)
test_data2D_paths_unnpacked = utils.unpack_2D_fpaths(test_data2D_paths)
# val_data2D_paths_unnpacked = utils.unpack_2D_fpaths(val_data2D_paths)

In [0]:
print(f"Train set has {len(train_data2D_paths_unnpacked)} samples")
print(f"Test set has {len(test_data2D_paths_unnpacked)} samples")
print(f"Validation set has {len(val_data2D_paths_unnpacked)} samples")

Train set has 15415 samples
Test set has 3234 samples
Validation set has 3381 samples


# Train models

### Create callbacks

In [0]:
reducer_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=4, min_lr=1e-6, verbose=1)

### Init train variables

In [0]:
history_path_3DUnet = '/content/drive/My Drive/VolumMedSeg/models/BraTS/UNet3D/Unet3D.h5'
history_path_Dilated3DUnet = '/content/drive/My Drive/VolumMedSeg/models/BraTS/UNet3DDilated/Unet3DDilated.h5'
history_path_2DUnet = '/content/drive/My Drive/VolumMedSeg/models/BraTS/UNet2D/Unet2D.h5'
history_path_Dilated2DUnet = '/content/drive/My Drive/VolumMedSeg/models/BraTS/UNet2DDilated/Unet2DDilated.h5'

In [0]:
BATCH_SIZE = 1
MERGE_CLASSES = True
SHUFFLE = True
AUGMENTATION = {'flip': False, 'rand_rot': True, 'hist_dist': False}
OUTPUT_CLASSES = ['ncr', 'et', 'ed']
SCAN_TYPES = ['t1', 't1ce', 't2', 'flair']
DIM3D = (144, 192, 160)
DIM2D = (192, 160)
EPOCHS = 50

n_classes = len(OUTPUT_CLASSES) if not MERGE_CLASSES else 1
n_channels = len(SCAN_TYPES)
input3D_shape = (*DIM3D, n_channels)
input2D_shape = (*DIM2D, n_channels)

custom_objects = {'soft_dice_loss': soft_dice_loss, 'mean_iou': mean_iou}

### Train 3D U-Nets

#### Create 3D data generators

In [0]:
train3D_params = {
          'dim': DIM3D,
          'batch_size': BATCH_SIZE,
          'merge_classes': MERGE_CLASSES,
          'shuffle': SHUFFLE,
          'flip': AUGMENTATION['flip'],
          'rand_rot': AUGMENTATION['rand_rot'],
          'hist_dist': AUGMENTATION['hist_dist'],
          'scan_types': SCAN_TYPES,
          'output_classes': OUTPUT_CLASSES,
          }

test3D_params = {
          'dim': DIM3D,
          'batch_size': BATCH_SIZE,
          'merge_classes': MERGE_CLASSES,
          'scan_types': SCAN_TYPES,
          'output_classes': OUTPUT_CLASSES,
          }


training3D_generator = Med3DDataGenerator(train_data3D_paths, **train3D_params)
testing3D_generator = Med3DDataGenerator(test_data3D_paths, **test3D_params)

#### Train 3D U-Net model

In [0]:
unet3d = Unet(n_classes=n_classes,
              input_shape=input3D_shape,
              n_base_filters=16,
              activation="elu",
              dropout_type="spatial",
              bottleneck_depth=2,
              mode="3D",
              )

model = unet3d.build_model()
model.compile(Adam(lr=1e-4), loss=soft_dice_loss)

In [0]:
resumable_model = ResumableModel(model, save_every_epochs=3, custom_objects=custom_objects, to_path=history_path_3DUnet)
history = resumable_model.fit(training3D_generator, validation_data=testing3D_generator, verbose=1, epochs=EPOCHS, callbacks=[reducer_lr])

#### Train Dilated 3D U-Net model

In [0]:
unet3d_dil = Unet(n_classes=n_classes,
              input_shape=input3D_shape,
              n_base_filters=16,
              activation="elu",
              dropout_type="spatial",
              dilate=True,
              mode="3D",
              )
model = unet3d_dil.build_model()

model.compile(Adam(lr=1e-4), loss=soft_dice_loss)

In [0]:
resumable_model = ResumableModel(model, save_every_epochs=3, custom_objects=custom_objects, to_path=history_path_Dilated3DUnet)
history = resumable_model.fit(training3D_generator, validation_data=testing3D_generator, verbose=1, epochs=EPOCHS, callbacks=[reducer_lr])

### Train 2D U-Nets

#### Create 2D data generators

In [0]:
train2D_params = {
          'dim': DIM2D,
          'batch_size': BATCH_SIZE,
          'merge_classes': MERGE_CLASSES,
          'shuffle': SHUFFLE,
          #'flip': AUGMENTATION['flip'],
          'rand_rot': AUGMENTATION['rand_rot'],
          'hist_dist': AUGMENTATION['hist_dist'],
          'scan_types': SCAN_TYPES,
          'output_classes': OUTPUT_CLASSES,
          }

test2D_params = {
          'dim': DIM2D,
          'batch_size': BATCH_SIZE,
          'merge_classes': MERGE_CLASSES,
          'scan_types': SCAN_TYPES,
          'output_classes': OUTPUT_CLASSES,
          }

training2D_generator = Med2DDataGenerator(train_data2D_paths_unnpacked, **train2D_params)
testing2D_generator = Med2DDataGenerator(test_data2D_paths_unnpacked, **test2D_params)

#### Train 2D U-Net model

In [0]:
unet2d = Unet(n_classes=n_classes,
              input_shape=input2D_shape,
              n_base_filters=16,
              activation="elu",
              dropout_type="spatial",
              bottleneck_depth=2,
              mode="2D"
              )
model = unet2d.build_model()

model.compile(Adam(lr=1e-4), loss=soft_dice_loss)

In [0]:
resumable_model = ResumableModel(model, save_every_epochs=3, custom_objects=custom_objects, to_path=history_path_2DUnet)
history = resumable_model.fit(training2D_generator, validation_data=testing2D_generator, verbose=1, epochs=EPOCHS, callbacks=[reducer_lr])

#### Train Dilated 2D U-Net model

In [0]:
unet2d_dil = Unet(n_classes=n_classes,
              input_shape=input2D_shape,
              n_base_filters=16,
              activation="elu",
              dropout_type="spatial",
              dilate=True,
              mode="2D",
              )
model = unet2d_dil.build_model()

model.compile(Adam(lr=1e-4), loss=soft_dice_loss)

In [0]:
resumable_model = ResumableModel(model, save_every_epochs=3, custom_objects=custom_objects, to_path=history_path_Dilated2DUnet)
history = resumable_model.fit(training2D_generator, validation_data=testing2D_generator, verbose=1, epochs=EPOCHS, callbacks=[reducer_lr])